In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL AWS

In [7]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo `csv` "CALLOUT"

In [3]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CALLOUT.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,submit_wardid,submit_careunit,curr_wardid,curr_careunit,callout_wardid,callout_service,request_tele,...,callout_status,callout_outcome,discharge_wardid,acknowledge_status,createtime,updatetime,acknowledgetime,outcometime,firstreservationtime,currentreservationtime
0,3917,10017,199207,7,NaN,45,CCU,1,MED,1,...,Inactive,Discharged,45.0,Acknowledged,2149-05-31 10:44:34,2149-05-31 10:44:34,2149-05-31 15:08:04,2149-05-31 22:40:02,NaN,NaN
1,3919,10026,103770,33,NaN,3,SICU,3,NMED,1,...,Inactive,Discharged,3.0,Revised,2195-05-18 13:56:20,2195-05-19 15:45:30,NaN,2195-05-19 17:40:03,NaN,NaN
2,3920,10027,199395,12,NaN,55,CSRU,55,CSURG,1,...,Inactive,Discharged,55.0,Acknowledged,2190-07-20 08:15:20,2190-07-20 08:15:20,2190-07-20 08:57:46,2190-07-20 17:10:02,NaN,NaN


In [4]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "CALLOUT" en la BD

In [8]:
sql_insert = ''' INSERT 
                   INTO  CALLOUT
                         (row_id, subject_id, hadm_id, submit_wardid, submit_careunit, curr_wardid, curr_careunit, 
                          callout_wardid, callout_service, request_tele, request_resp, request_cdiff, request_mrsa,
                          request_vre, callout_status, callout_outcome, discharge_wardid, acknowledge_status, createtime,
                          updatetime, acknowledgetime, outcometime, firstreservationtime, currentreservationtime)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    subw = int(df.loc[i, 'submit_wardid'])
    subc = df.loc[i, 'submit_careunit']
    curw = int(df.loc[i, 'curr_wardid'])
    curc = df.loc[i, 'curr_careunit']
    couw = int(df.loc[i, 'callout_wardid'])
    cous = df.loc[i, 'callout_service']
    retl = int(df.loc[i, 'request_tele'])
    rers = int(df.loc[i, 'request_resp'])
    recd = int(df.loc[i, 'request_cdiff'])
    mrsa = int(df.loc[i, 'request_mrsa'])
    revr = int(df.loc[i, 'request_vre'])
    cost = df.loc[i, 'callout_status']
    cout = df.loc[i, 'callout_outcome']
    disw = int(df.loc[i, 'discharge_wardid'])
    akst = df.loc[i, 'acknowledge_status']
    crtm = df.loc[i, 'createtime']
    uptm = df.loc[i, 'updatetime']
    aktm = df.loc[i, 'acknowledgetime']
    outm = df.loc[i, 'outcometime']
    frtm = df.loc[i, 'firstreservationtime']
    curt = df.loc[i, 'currentreservationtime']
    reg_s = (row, subj, hadm, subw, subc, curw, curc, couw, cous, retl, rers, recd, mrsa, revr, cost, cout, disw, akst, crtm,
             uptm, aktm, outm, frtm, curt)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0


### Confirma la inserción de los registros a la BD

In [9]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [10]:
cursor.close()
con.close()